# MLP for timeseries example

In [ ]:
from sam.models import TimeseriesMLP
from sam.feature_engineering import SimpleFeatureEngineer

import pandas as pd

In [ ]:
data = pd.read_parquet("../data/rainbow_beach.parquet")
data.head()

In [ ]:
simple_features = SimpleFeatureEngineer(
    rolling_features=[
        ("wave_height", "mean", 48),
        ("wave_height", "mean", 24),
        ("wave_height", "mean", 12),
    ],
    time_features=[
        ("hour_of_day", "cyclical"),
        ("day_of_week", "cyclical"),
    ],
    keep_original=False,
)

X = data
y = data['water_temperature']


In [ ]:
model = TimeseriesMLP(
    predict_ahead=(0,),
    feature_engineer=simple_features,
    use_diff_of_y=False,
    epochs=20,
    verbose=0
)

model.fit(X, y)

In [ ]:
model.predict(X, y).plot()
y.plot()

## Using a pipeline for feature engineering

Timeseries models should support any sklearn pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

simple_features = SimpleFeatureEngineer(
    rolling_features=[
        ("wave_height", "mean", 48),
        ("wave_height", "mean", 24),
        ("wave_height", "mean", 12),
    ],
    time_features=[
        ("hour_of_day", "cyclical"),
        ("day_of_week", "cyclical"),
    ],
    keep_original=False,
)

feature_pipeline = Pipeline(steps=[
    ('feature_engineer', simple_features),
    ('imputer', SimpleImputer(strategy='mean')),
])


model = TimeseriesMLP(
    predict_ahead=(0,),
    feature_engineer=feature_pipeline,
    use_diff_of_y=False,
    epochs=20,
    verbose=0
)

model.fit(X, y)

In [ ]:
model.predict(X, y).plot()